In [1]:
import mdtraj as md
import numpy as np
import MDAnalysis as mda

In [2]:
# 6Y2E as reference structure.
def translate(xyz, displacement):
    """
    translate the ligand by displacement in nanometers
    """
    for atom_ind in range(xyz.shape[1]):
        xyz[0,atom_ind,:] += displacement
    return None

# stolen from MDAnalysis
def rotate(ref, R, point=(0, 0, 0)):
    R = np.asarray(R)
    # point = np.asarray(point)
    point = np.mean(ref.xyz[0],axis=(0))
    

    # changes the coordinates (in place)
    # translate(ref.xyz, -point)
    x = ref.xyz[0]
    idx = ref.topology.select("all")
    x[idx] = np.dot(x, R.T)
    translate(ref.xyz, point)

    return ref
    

In [3]:
def generate_structures(pdb, ref):
    import MDAnalysis.analysis.align as align
    import copy
    R = np.array([[-1.0, 0.0, 0.0],
              [ 0.0, 1.0, 0.0],
              [ 0.0, 0.0,-1.0]])
    T = np.array([-8.72884, 0.0, 43.91490])
    u = mda.Universe(pdb)
    u2 = mda.Universe(pdb)
    ref = mda.Universe(ref)
    refA = ref.select_atoms("resid 0 to 305")
    A = u.select_atoms("chainID A")
    A_flip = u2.select_atoms("chainID A")
    align.alignto(A, refA, select="protein and name CA", weights="mass")
    align.alignto(A_flip, refA, select="protein and name CA", weights="mass")
    AB = u.select_atoms("all")
    AB.write("test/AB_original.pdb")
    # A = u.select_atoms("index 0 to 4681")

    A.write("test/A_original.pdb")
    B = u.select_atoms("index 4682 to 9364")
    B.write("test/B_original.pdb")
    # A.rotate(((-1,0,0),(0,1,0),(0,0,-1)), point=(-8.72884, 0.0, 43.91490))
    # B.rotate(((-1,0,0),(0,1,0),(0,0,-1)), point=(-8.72884, 0.0, 43.91490))
    # A_flip = u.select_atoms("chainID A")
    B_flip = u2.select_atoms("index 4682 to 9364")
    A_flip.atoms.rotate(R)
    A_flip.atoms.translate(T)
    B_flip.atoms.rotate(R)
    B_flip.atoms.translate(T)
    
    AA = mda.Merge(A,A_flip)
    BB = mda.Merge(B,B_flip)
    BA = mda.Merge(A_flip, B_flip)
    # AA.write("test/AA.pdb")
    # BB.write("test/BB.pdb")
    # AB.write("test/AB_flip.pdb")
    return AA, BB, BA

In [4]:
import glob
files = glob.glob('cluster_centers/*.pdb')
files.sort()

In [ ]:
files

In [11]:
import re
structs = {}
files.sort()
for file in files:
    match = re.search(r'\d+', file)
    if match:
        number = int(match.group())
        padded_number = str(number).zfill(4)
        structs[padded_number] = file
structs = {k: structs[k] for k in sorted(structs)}
    

In [13]:
names = list(structs.keys())
results = []
for name in names:
    pdb = structs[name]
    aa,bb,ba = generate_structures(pdb, "6y2e.pdb")
    aa.atoms.write(f'cluster_center_results/{name}_AA.pdb')
    bb.atoms.write(f'cluster_center_results/{name}_BB.pdb')
    ba.atoms.write(f'cluster_center_results/{name}_BA.pdb')   
    

/home/jim/anaconda3/envs/FFTp39/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/home/jim/anaconda3/envs/FFTp39/lib/python3.9/site-packages/MDAnalysis/coordinates/PDB.py:749: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "


In [45]:
import MDAnalysis.analysis.align as align
u = mda.Universe("selected_frs_TS0_1_black.pdb")
ref = mda.Universe("6y2e.pdb")
refA = ref.select_atoms("resid 0 to 305")
A = u.select_atoms("chainID A")
align.alignto(A, refA, select="protein and name CA", weights="mass")
AB = u.select_atoms("all")
AB.write("test/AB_original.pdb")
# A = u.select_atoms("index 0 to 4681")

A.write("test/A_original.pdb")
B = u.select_atoms("index 4682 to 9364")
B.write("test/B_original.pdb")
# A.rotate(((-1,0,0),(0,1,0),(0,0,-1)), point=(-8.72884, 0.0, 43.91490))
# B.rotate(((-1,0,0),(0,1,0),(0,0,-1)), point=(-8.72884, 0.0, 43.91490))
A.atoms.rotate(R)
A.atoms.translate(T)
B.atoms.rotate(R)
B.atoms.translate(T)

A.write("test/A_flip.pdb")
B.write("test/B_flip.pdb")
AB.write("test/AB_flip.pdb")

In [34]:
refA

<AtomGroup with 306 atoms>

In [160]:
flipped.save_pdb('pink_flipped_test.pdb')
ap.save_pdb('a_test.pdb')
bp.save_pdb('b_test.pdb')
# a_copy.save_pdb('a_copy_test.pdb')
# b_copy.save_pdb('b_copy_test.pdb')

In [168]:
df = flipped.topology.to_dataframe()

In [178]:
df[1]

array([[4.000e+00, 1.100e+01, 0.000e+00, 0.000e+00],
       [1.100e+01, 1.200e+01, 0.000e+00, 0.000e+00],
       [4.000e+00, 6.000e+00, 0.000e+00, 0.000e+00],
       ...,
       [9.346e+03, 9.347e+03, 0.000e+00, 0.000e+00],
       [9.358e+03, 9.359e+03, 0.000e+00, 0.000e+00],
       [9.358e+03, 9.360e+03, 0.000e+00, 0.000e+00]])

In [11]:
print(flipped.xyz.sum())
R = [ [-1.0, 0.0, 0.0],[ 0.0, 1.0, 0.0],[ 0.0, 0.0,-1.0]]
T = [-8.72884, 0.00, 43.91490]
test = rotate(flipped, R, T)
print(flipped.xyz.sum(), test.xyz.sum())

NameError: name 'flipped' is not defined

In [ ]:
# original rotation and transform tests that didn't work due to lack of alignment to xtal

In [ ]:
import numpy as np

u = mda.Universe("selected_frs_TS0_1_black.pdb")

# define rotation matrix and translation vector
rot_mat = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])
trans_vec = np.array([-8.72884, 0.0, 43.91490])

from MDAnalysis.transformations import rotate, translate

# apply rotation matrix
rotate.rotation_matrix(u.atoms.positions, rot_mat)

# apply translation vector
translate(u.atoms.positions, trans_vec)

from MDAnalysis.coordinates import PDBWriter

# save transformed structure to new PDB file
with PDBWriter("results/transformed_protein.pdb") as writer:
    writer.write(u.atoms)


In [9]:
u = mda.Universe("selected_frs_TS0_1_black.pdb")

rot_mat1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
trans_vec1 = np.array([0.0, 0.0, 0.0])
trans_mat1 = np.eye(4)
trans_mat1[:3, :3] = rot_mat1
trans_mat1[:3, 3] = trans_vec1
trans_mat1[3, :] = [0, 0, 0, 1]

rot_mat2 = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])
trans_vec2 = np.array([-8.72884, 0.0, 43.91490])
trans_mat2 = np.eye(4)
trans_mat2[:3, :3] = rot_mat2
trans_mat2[:3, 3] = trans_vec2
trans_mat2[3, :] = [0, 0, 0, 1]


s = u.select_atoms("index 0 to 4681")
s.write("results/A_reg.pdb")
s.transform(trans_mat1)
s.write("results/A_mat1.pdb")
s.transform(trans_mat2)
s.write("results/A_flip.pdb")

In [ ]:
# Non working original method using alignment...

In [3]:
def flip_protomers(ref):
    import copy
    ref2 = copy.copy(ref)
    a = ref.topology.select("index 0 to 4681")
    b = ref.topology.select("index 4682 to 9364")
    top_a = ref.topology.subset(a)
    top_b = ref.topology.subset(b)
    traj_a = md.Trajectory(ref.xyz[0,0:4682,:], top_a)
    traj_a_copy = copy.deepcopy(traj_a)
    traj_b = md.Trajectory(ref.xyz[0,4682::,:], top_b)
    traj_b_copy = copy.deepcopy(traj_b)
    traj_a.superpose(traj_b_copy,frame=0,atom_indices=a,ref_atom_indices=a)
    traj_b.superpose(traj_a_copy,frame=0,atom_indices=a,ref_atom_indices=a)
    # traj_a.superpose(traj_b_copy,frame=0)
    # traj_b.superpose(traj_a_copy,frame=0)
    xyz = np.concatenate((traj_a.xyz,traj_b.xyz), axis=1)
    # print(xyz.shape, xyz)
    pdb = md.Trajectory(xyz, ref.topology)
    
    return pdb, traj_a, traj_b

In [4]:
def double_A(ref):
    import copy
    ref2 = copy.deepcopy(ref)
    a1 = ref.topology.select("index 0 to 4681")
    a2 = ref2.topology.select("index 0 to 4681")
    b = ref.topology.select("index 4682 to 9364")
    top_a1 = ref.topology.subset(a1)
    top_a2 = ref.topology.subset(a2)
    top_b = ref.topology.subset(b)
    traj_a1 = md.Trajectory(ref.xyz[0,0:4682,:], top_a1)
    traj_a1_copy = copy.deepcopy(traj_a1)
    traj_a2 = md.Trajectory(ref.xyz[0,0:4682,:], top_a2)
    traj_a2_copy = copy.deepcopy(traj_a2)
    traj_b = md.Trajectory(ref.xyz[0,4682::,:], top_b)
    traj_b_copy = copy.deepcopy(traj_b)
    # print(traj_a1.xyz.sum(), traj_a2.xyz.sum(), traj_b.xyz.sum(), ref2.xyz[0,0:4682,:].sum())
    traj_a2.superpose(traj_b_copy,frame=0,atom_indices=a1,ref_atom_indices=a2)
    # print(traj_a1.xyz.sum(), traj_a2.xyz.sum(), traj_b.xyz.sum(), ref2.xyz[0,0:4682,:].sum())
    xyz = np.concatenate((traj_a1.xyz,ref2.xyz[:,0:4682,:]), axis=1)
    # print(xyz.shape, xyz)
    traj_a_test = md.Trajectory(ref2.xyz[0,0:4682,:], ref2.topology.subset(a2))
    # print(traj_a_test.xyz.sum())
    pdb = md.Trajectory(xyz, ref.topology)
    
    return pdb, traj_a2, traj_a_test

In [5]:
def double_B(ref):
    import copy
    ref2 = copy.deepcopy(ref)
    a = ref.topology.select("index 0 to 4681")
    b1 = ref.topology.select("index 4682 to 9364")
    b2 = ref2.topology.select("index 4682 to 9364")
    top_b1 = ref.topology.subset(b1)
    top_b2 = ref.topology.subset(b2)
    top_a = ref.topology.subset(a)
    traj_b1 = md.Trajectory(ref.xyz[0,4682::,:], top_b1)
    traj_b1_copy = copy.deepcopy(traj_b1)
    traj_b2 = md.Trajectory(ref.xyz[0,4682::,:], top_b2)
    traj_b2_copy = copy.deepcopy(traj_b2)
    traj_a = md.Trajectory(ref.xyz[0,0:4682,:], top_a)
    traj_a_copy = copy.deepcopy(traj_a)
    # print(traj_b1.xyz.sum(), traj_b2.xyz.sum(), traj_a.xyz.sum(), ref2.xyz[0,4682::,:].sum())
    traj_b2.superpose(traj_a_copy,frame=0,atom_indices=a,ref_atom_indices=a)
    # print(traj_b1.xyz.sum(), traj_b2.xyz.sum(), traj_a.xyz.sum(), ref2.xyz[0,4682::,:].sum())
    # print(traj_b1.xyz.shape,ref2.xyz[0,4682::,:].shape)
    xyz = np.concatenate((traj_b1.xyz,ref2.xyz[:,4682::,:]), axis=1)
    # print(xyz.shape, xyz)
    traj_b_test = md.Trajectory(ref2.xyz[:,4682::,:], ref2.topology.subset(b2))
    # print(traj_b_test.xyz.sum())
    pdb = md.Trajectory(xyz, ref.topology)
    
    return pdb, traj_b2, traj_b_test

In [6]:
def make_double_protomers(ref):
    import copy
    ref2 = copy.deepcopy(ref)
    ref3 = copy.deepcopy(ref)
    ref4 = copy.deepcopy(ref)
    a1 = ref.topology.select("index 0 to 4681")
    a2 = ref2.topology.select("index 0 to 4681")
    b = ref.topology.select("index 4682 to 9364")
    top_a1 = ref.topology.subset(a1)
    top_a2 = ref.topology.subset(a2)
    top_b = ref.topology.subset(b)
    traj_a1 = md.Trajectory(ref.xyz[0,0:4682,:], top_a1)
    traj_a1_copy = copy.deepcopy(traj_a1)
    traj_a2 = md.Trajectory(ref.xyz[0,0:4682,:], top_a2)
    traj_a2_copy = copy.deepcopy(traj_a2)
    traj_b = md.Trajectory(ref.xyz[0,4682::,:], top_b)
    traj_b_copy = copy.deepcopy(traj_b)
    a = ref3.topology.select("index 0 to 4681")
    b1 = ref3.topology.select("index 4682 to 9364")
    b2 = ref4.topology.select("index 4682 to 9364")
    top_b1 = ref3.topology.subset(b1)
    top_b2 = ref3.topology.subset(b2)
    top_a = ref3.topology.subset(a)
    traj_b1 = md.Trajectory(ref3.xyz[0,4682::,:], top_b1)
    traj_b1_copy = copy.deepcopy(traj_b1)
    traj_b2 = md.Trajectory(ref3.xyz[0,4682::,:], top_b2)
    traj_b2_copy = copy.deepcopy(traj_b2)
    traj_a = md.Trajectory(ref3.xyz[0,0:4682,:], top_a)
    traj_a_copy = copy.deepcopy(traj_a)
    # traj_a.superpose(traj_b_copy,frame=0,atom_indices=a,ref_atom_indices=a)
    # traj_b.superpose(traj_a_copy,frame=0,atom_indices=a,ref_atom_indices=a)
    rotate(traj_a1_copy, ((-1,0,0),(0,1,0),(0,0,-1)), (-.872884, 0.0, 4.391490))
    rotate(traj_b1_copy, ((-1,0,0),(0,1,0),(0,0,-1)), (-.872884, 0.0, 4.391490))
    # traj_a.superpose(traj_b_copy,frame=0)
    # traj_b.superpose(traj_a_copy,frame=0)
    xyz_aa = np.concatenate((traj_a1_copy.xyz,ref2.xyz[:,0:4682,:]), axis=1)
    xyz_bb = np.concatenate((traj_b1_copy.xyz,ref4.xyz[:,4682::,:]), axis=1)
    xyz_ba = np.concatenate((traj_b1_copy.xyz,traj_a1_copy.xyz), axis=1)
    # print(xyz.shape, xyz)
    pdb_aa = md.Trajectory(xyz_aa, ref.topology)
    pdb_bb = md.Trajectory(xyz_bb, ref.topology)
    pdb_ba = md.Trajectory(xyz_ba, ref.topology)
    
    
    return pdb_aa, pdb_bb, pdb_ba, traj_a, traj_b